<h2>Big Data Systems and Architectures - Spark Assignment 2021</h2>
<h3>Exploring International Flights in 2017 Data</h3>

---
> Georgia Vlassi p2822001<br />
> Business Analytics <br />
> Athens University of Economics and Business <br/>

---

The main scope of this assignment is to analyse a dataset about international flights in 2017, using Apache Spark to reveal insights about these data. 

You can find the aforementioned data here:
http://andrea.imis.athena-innovation.gr/aueb-master/flights.csv.zip


<h3>Task 2</h3>
The objective is to create reports on the 
average and median departure delays of (a) all the airports(ORIGIN), and (b) all the airways(CARRIER)in the 
dataset. You should give four reports, two for the airports (average/median delays) and two 
for the airways (average/median delays). Each report is a CSV file containing one line for each 
airport/airway and the lines of each file should be ordered (in descending order) based on the 
corresponding criterion (average/median delay). No header files are required for these files. 
An extra instruction you have from your supervisor is that you should take care of some data 
outliers: you should not consider in your analysis any airports/airways that have extremely 
low number of flights; the criterion is that any airport/airway belonging in the lowest 1% 
percentile, regarding the number of flights, should be omitted.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.readwriter import DataFrameReader
from pyspark.sql.types import IntegerType, Row, StringType

from pyspark.sql.window import Window
from pyspark.sql.functions import col,percent_rank,count,avg,expr,desc,round

Our first step is to create a temporary view to read the data.

In [2]:
spark = SparkSession.builder.appName("FlightsAssignment").getOrCreate()
spark

After initialising the view we have to load our data. The data file named '671009038_T_ONTIME_REPORTING.csv', will be read by using using spark and save them into flights_data variable. In order to access the data download the file mentioned on description and unzip it.

In [3]:
flights_data = spark.read\
                    .option("header","true")\
                    .option("inferSchema","true")\
                    .csv("671009038_T_ONTIME_REPORTING.csv")

In [4]:
flights_data.show(2)

+----------+--------+-------+------+----------------+----+------------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----+
|   FL_DATE|TAIL_NUM|CARRIER|ORIGIN|ORIGIN_CITY_NAME|DEST|    DEST_CITY_NAME|DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c19|
+----------+--------+-------+------+----------------+----+------------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----+
|2019-01-01|  N8974C|     9E|   AVL|   Asheville, NC| ATL|       Atlanta, GA|    1658|     -7.0|    1758|    -22.0|      0.0|             null|     0.0|         null|         null|     null|          null|               null|null|
|2019-01-01|  N922XJ|     9E|   JFK|    New York, NY| RDU|Raleigh/Durham, NC

In [5]:
flights_data.count()

7422037

Our dataset should be cleaned. As mentioned, there are airports, which have very few flights. More specific we have to omit the airports in the lowest 1% percentile in the number of flights.

In the beginning, we will count the percentiles of the flights each airport has.

<h4>ORIGIN</h4>

In [6]:
#calculate percentiles
percentiles_ap = flights_data.groupBy("ORIGIN").count()\
                .select("ORIGIN","count", percent_rank().over(Window.partitionBy().orderBy("count")).alias("percent_rank_ap"))
                
percentiles_ap.show(5)

+------+-----+--------------------+
|ORIGIN|count|     percent_rank_ap|
+------+-----+--------------------+
|   AKN|   61|                 0.0|
|   PGV|   80|0.002785515320334262|
|   DLG|   82|0.005571030640668524|
|   GST|   82|0.005571030640668524|
|   HYA|   83|0.011142061281337047|
+------+-----+--------------------+
only showing top 5 rows



Count before removing outliers

In [7]:
flights_before = flights_data.count()
flights_before

7422037

In [8]:
#join percentiles to initial dataset
flights_percentiles_ap = flights_data.alias('flights').join(percentiles_ap.alias('percentiles'), col('percentiles.ORIGIN') == col('flights.ORIGIN'))\
            .select([col('flights.'+xx) for xx in flights_data.columns] + [col('percentiles.percent_rank_ap')])

flights_percentiles_ap.show(5)

+----------+--------+-------+------+----------------+----+--------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----+-------------------+
|   FL_DATE|TAIL_NUM|CARRIER|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c19|    percent_rank_ap|
+----------+--------+-------+------+----------------+----+--------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----+-------------------+
|2019-01-01|  N629BR|     OO|   BGM|  Binghamton, NY| DTW|   Detroit, MI|    1743|     -4.0|    1940|      9.0|      0.0|             null|     0.0|         null|         null|     null|          null|               null|null|0.24512534818941503|
|2019-01-01|

In [9]:
#remove outliers (aka < 0.01) and store the rest on a new dataframe
flights_data = flights_percentiles_ap.where("percent_rank_ap > 0.01")

Count after removing outlers

In [10]:
flights_after_ap = flights_data.count()
flights_after_ap

7421732

For each airport calculate the average DEP_DELAY.

In [11]:
from pyspark.sql.functions import avg,expr, col, desc,round

avgDelayPerAirport = flights_data\
               .groupBy("ORIGIN")\
               .agg(round(avg(("DEP_DELAY")),2).alias("AverageDelay"))\
               .sort(desc("AverageDelay"))
avgDelayPerAirport.show()

+------+------------+
|ORIGIN|AverageDelay|
+------+------------+
|   OTH|       33.78|
|   XWA|        32.6|
|   MMH|       30.97|
|   HYA|       29.35|
|   MEI|       28.88|
|   ACK|       28.15|
|   EGE|       26.46|
|   MQT|       26.19|
|   HGR|       25.18|
|   CMX|       24.05|
|   ACV|       23.68|
|   SHD|       23.59|
|   OGS|       23.41|
|   OGD|        23.3|
|   ASE|       23.24|
|   SLN|       22.85|
|   SWF|       22.18|
|   BLV|       21.48|
|   CKB|       21.41|
|   STC|       21.32|
+------+------------+
only showing top 20 rows



In [12]:
avgDelayPerAirport.limit(100).coalesce(1)\
       .write.csv("task2-ap-avg")

For each airport calculate the median DEP_DELAY.

In [13]:
medDelayPerAirport = flights_data\
                .groupBy("ORIGIN")\
                .agg(expr('percentile_approx(DEP_DELAY, 0.5)').alias("MedianDelay"))\
                .sort(desc("MedianDelay"))
medDelayPerAirport.show()

+------+-----------+
|ORIGIN|MedianDelay|
+------+-----------+
|   ADK|        8.0|
|   OGD|        7.0|
|   HYA|        3.0|
|   PPG|        3.0|
|   MDW|        2.0|
|   DAL|        1.0|
|   HOU|        1.0|
|   LCK|        0.0|
|   BLV|        0.0|
|   BWI|        0.0|
|   AZA|        0.0|
|   OAK|        0.0|
|   XWA|        0.0|
|   ART|        0.0|
|   HGR|        0.0|
|   SCK|        0.0|
|   STL|        0.0|
|   HTS|        0.0|
|   PSM|       -1.0|
|   MSY|       -1.0|
+------+-----------+
only showing top 20 rows



In [14]:
medDelayPerAirport.limit(100).coalesce(1)\
       .write.csv("task2-ap-med")

Our dataset should be cleaned. As mentioned, there are airports, which have very few flights. More specific we have to omit the airports in the lowest 1% percentile in the number of flights.

In the beginning, we will count the percentiles of the flights each airway has.

<h4>CARRIER</h4>

In [15]:
#calculate percentiles
percentiles_aw = flights_data.groupBy("CARRIER").count()\
                .select("CARRIER","count", percent_rank().over(Window.partitionBy().orderBy("count")).alias("percent_rank_aw"))
                
percentiles_aw.show(5)

+-------+------+---------------+
|CARRIER| count|percent_rank_aw|
+-------+------+---------------+
|     HA| 83891|            0.0|
|     G4|105305|         0.0625|
|     EV|134683|          0.125|
|     F9|135543|         0.1875|
|     NK|204845|           0.25|
+-------+------+---------------+
only showing top 5 rows



In [16]:
flights_after_aw = flights_data.count()
flights_after_aw

7421732

In [17]:
#join percentiles_aw to initial dataset
flights_percentiles_aw = flights_data.alias('flights').join(percentiles_aw.alias('percentiles'), col('percentiles.CARRIER') == col('flights.CARRIER'))\
            .select([col('flights.'+xx) for xx in flights_data.columns] + [col('percentiles.percent_rank_aw')])

flights_percentiles_aw.show(5)

+----------+--------+-------+------+----------------+----+-----------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----+------------------+---------------+
|   FL_DATE|TAIL_NUM|CARRIER|ORIGIN|ORIGIN_CITY_NAME|DEST|   DEST_CITY_NAME|DEP_TIME|DEP_DELAY|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c19|   percent_rank_ap|percent_rank_aw|
+----------+--------+-------+------+----------------+----+-----------------+--------+---------+--------+---------+---------+-----------------+--------+-------------+-------------+---------+--------------+-------------------+----+------------------+---------------+
|2019-01-01|  N484UA|     UA|   MSY| New Orleans, LA| DEN|       Denver, CO|    1608|      1.0|    1758|     -3.0|      0.0|             null|     0.0|         null|         null|     null|          null| 

In [18]:
#remove outliers (aka < 0.01) and store the rest on a new dataframe
flights_data = flights_percentiles_aw.where("percent_rank_aw > 0.01")

In [19]:
flights_after_aw = flights_data.count()
flights_after_aw

7337841

Calculate with the initial dataset

In [20]:
from pyspark.sql.functions import avg,expr, col, desc, round

avgDelayPerAirway = flights_data\
               .groupBy("CARRIER")\
               .agg(round(avg(("DEP_DELAY")),2).alias("AverageDelay"))\
               .sort(desc("AverageDelay"))
avgDelayPerAirway.show()

+-------+------------+
|CARRIER|AverageDelay|
+-------+------------+
|     B6|       17.75|
|     EV|       17.21|
|     F9|       14.58|
|     YV|        13.8|
|     UA|        13.0|
|     OO|       12.56|
|     AA|       12.11|
|     NK|       10.94|
|     OH|        10.7|
|     9E|       10.25|
|     WN|       10.18|
|     G4|       10.12|
|     MQ|        9.27|
|     YX|        8.54|
|     DL|        8.16|
|     AS|        5.04|
+-------+------------+



Calculate with the dataset omitted outliers (SAME results) - No need to check outliers

In [21]:
avgDelayPerAirway.limit(100).coalesce(1)\
       .write.csv("task2-aw-avg")

In [22]:
medDelayPerAirway = flights_data\
                .groupBy("CARRIER")\
                .agg(expr('percentile_approx(DEP_DELAY, 0.5)').alias("MedianDelay"))\
                .sort(desc("MedianDelay"))
medDelayPerAirway.show()

+-------+-----------+
|CARRIER|MedianDelay|
+-------+-----------+
|     WN|        0.0|
|     AA|       -2.0|
|     DL|       -2.0|
|     B6|       -3.0|
|     NK|       -3.0|
|     OH|       -3.0|
|     MQ|       -3.0|
|     UA|       -3.0|
|     G4|       -3.0|
|     F9|       -3.0|
|     YV|       -3.0|
|     OO|       -3.0|
|     EV|       -4.0|
|     AS|       -4.0|
|     9E|       -4.0|
|     YX|       -4.0|
+-------+-----------+



In [23]:
medDelayPerAirway.limit(100).coalesce(1)\
       .write.csv("task2-aw-med")